## install bpy
- [參考網站](https://www.zhihu.com/question/386776864)
- steps
    1. `conda create -n ***`
    2. 至 https://pypi.org/project/bpy/#files 查看 python 版本 (cp***)
    3. 安裝對應 python 版本 `conda install python=***`
    4. `pip install bpy`
- (記得切換 kernel -> `conda activate bpy`)
## steps and references
- preprocessing
    - [convert curve to mesh](https://blender.stackexchange.com/questions/265215/how-can-i-convert-a-curve-to-a-mesh-object)
    - [get vertices from mesh](https://blender.stackexchange.com/questions/1311/how-can-i-get-vertex-positions-from-a-mesh)
    - [get original colors](https://blenderartists.org/t/svg-import-remove-redundant-materials/693325/4)
- create mesh
    - [triangulate polygon in shapely](https://stackoverflow.com/questions/65019170/how-do-you-triangulate-a-polygon-in-shapely)
- simplification
    - ~~[tripy](https://github.com/linuxlewis/tripy)~~
    - ~~[optimization](https://github.com/meshpro/optimesh)~~
    - [triangulate](https://github.com/lionfish0/earclip)
## other packages
- `pip install scipy`
- `pip install shapely`
- `pip install geopandas`
- `pip install geovoronoi`
- `pip install tripy`
- `pip install optimesh`

In [ ]:
import numpy as np
import bpy
import matplotlib.pyplot as plt
import shapely.wkt
import geopandas as gpd
import tripy
import optimesh
import warnings
import torch
import colorsys

from shapely.geometry import Polygon
from matplotlib.lines import Line2D
from geovoronoi import voronoi_regions_from_coords
from quad_mesh_simplify import simplify_mesh
from collections import OrderedDict
from utils.earclip import triangulate
from torch_geometric.data import Data

In [ ]:
# import file
file_path = "./datasets/svg/011-library.svg"
# file_path = "./datasets/svg/027-diamond.svg"
# file_path = "./datasets/svg/019-watermelon.svg"
# file_path = "./datasets/svg/024-book.svg"
# file_path = "./datasets/svg/032-firewood.svg"
# file_path = "./datasets/svg/037-time.svg"
file_path = "./datasets/svg/cherry-230123-1.svg"

# clean the scene
bpy.ops.object.select_all()
bpy.ops.object.delete()

bpy.ops.import_curve.svg(filepath=file_path)

In [ ]:
# convert curve to mesh
idx = 0
for ob in bpy.data.objects:
    if ob.type == "CURVE":
        mesh = bpy.data.meshes.new_from_object(ob)
        new_obj = bpy.data.objects.new("mesh_obj" + str(idx), mesh)
        new_obj.matrix_world = ob.matrix_world
        bpy.context.collection.objects.link(new_obj)
        idx += 1

In [ ]:
# get all curve meshes
colors = {}
nodes = {}
idx = 0
hsv = {}
for ob in bpy.data.objects:
    if ob.type == "MESH" and "mesh_obj" in ob.name:
        try:
            # get mesh
            rgb = ob.material_slots[0].material.diffuse_color
            colors[idx] = np.array([rgb[0], rgb[1], rgb[2]])
            hsv[idx] = np.array(colorsys.rgb_to_hsv(rgb[0], rgb[1], rgb[2]))
            
            # get vertices
            v = ob.data.vertices[0]
            coords = [(ob.matrix_world @ v.co) for v in ob.data.vertices]  # (x, y, z)
            nodes[idx] = []
            for x, y, z in coords:
                nodes[idx].append([x, y]) 
            nodes[idx] = np.array(nodes[idx])
            
            idx += 1
        except:
            continue

In [ ]:
# triangulate
polys = {}
for i, node in nodes.items():
    # polys[i] = np.array(tripy.earclip(node))
    polys[i] = np.array(triangulate(node.tolist()))

In [ ]:
# create mesh points and cells numpy array
pos_to_idx = {}
idx_to_pos = {}
mesh_faces = {}
mesh_points = {}

for i, poly_list in polys.items():
    cnt = 0
    pos_to_idx[i] = {}
    idx_to_pos[i] = {}
    mesh_faces[i] = []
    
    for pos in poly_list:
        for p in pos:
            if tuple(p) not in pos_to_idx[i]:
                pos_to_idx[i][tuple(p)] = cnt
                cnt += 1
        mesh_faces[i].append([pos_to_idx[i][tuple(pos[0])], pos_to_idx[i][tuple(pos[1])], pos_to_idx[i][tuple(pos[2])]])
    
    idx_to_pos[i] = {v: k for k, v in pos_to_idx[i].items()}
    mesh_faces[i] = np.array(mesh_faces[i])
    
for i, d in idx_to_pos.items():
    mesh_points[i] = []
    for v in d.values():
        mesh_points[i].append(list(v))
    mesh_points[i] = np.array(mesh_points[i])

In [ ]:
# mesh simplification
fig = plt.figure("plot")
ax = fig.add_subplot(1, 1, 1)
plt.axis("off")

new_points = []
new_colors = []
new_edges = []
idx = 0
out_points = {}

for i, (ori_points, ori_cells) in enumerate(zip(mesh_points.values(), mesh_faces.values())):
    if (len(ori_points) == 0): continue
    
    z = np.zeros([len(ori_points), 1])
    ori_points = np.concatenate((ori_points, z), axis=1)
    ori_cells = ori_cells.astype(np.uint32)
    
    warnings.filterwarnings("ignore")
      
    points, cells = simplify_mesh(ori_points, ori_cells, 10, max_err=0.2)
    points = points[:,:2]
    
    # points, cells = optimesh.optimize_points_cells(
    #     points, cells, "cpt-quasi-newton", 1.0e-5, 100
    # )
    
    # build points and colors
    out_points[idx] = points
    c_temp = [colors[i]] * len(points)
    if idx == 0:
        new_points = points
        new_colors = c_temp
    else:
        new_points = np.concatenate((new_points, points), axis=0)
        new_colors = np.concatenate((new_colors, c_temp), axis=0)
    
    # add edges
    for a, b, c in cells:
        new_edges.append([a+idx, b+idx])
        new_edges.append([b+idx, c+idx])
        new_edges.append([c+idx, a+idx])
    idx += len(points)
    
    print(f"cluster {i}:\n\
          number of original/new points = {len(ori_points)} / {len(points)}\n\
          number of original/new faces = {len(ori_cells)} / {len(cells)}")
    
    plt.scatter(points[:,0], points[:,1], color=colors[i], s=10, label=i)
    for a, b, c in cells:
        x1, y1 = points[a]
        x2, y2 = points[b]
        x3, y3 = points[c]
        l = Line2D([x1,x2], [y1,y2], color=colors[i], alpha=0.5)
        ax.add_line(l)
        l = Line2D([x1,x3], [y1,y3], color=colors[i], alpha=0.5)
        ax.add_line(l)
        l = Line2D([x3,x2], [y3,y2], color=colors[i], alpha=0.5)
        ax.add_line(l)

new_points = np.array(new_points)        
new_colors = np.array(new_colors)
new_edges = np.array(new_edges)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.show()

In [ ]:
# create pyg data
# nodes
n = new_points.shape[0]
x = np.zeros([n, 5]).astype(np.float32)
y = np.zeros([n, 3]).astype(np.float32)
for i, (points, colors) in enumerate(zip(new_points, new_colors)):
    y[i, :] = colors
    
    f = [0, 0, 0]
    f += points.tolist()
    x[i, :] = f
    
x = torch.Tensor(x)
y = torch.Tensor(y)

# edges
m = new_edges.shape[0]
edges = np.zeros([2*m, 2]).astype(np.int64)
for e, (s,t) in enumerate(new_edges):
    edges[e, 0] = s
    edges[e, 1] = t
    edges[m+e, 0] = t
    edges[m+e, 1] = s
edges = torch.Tensor(np.transpose(edges)).type(torch.long)

In [ ]:
data = Data(x=x, edge_index=edges, y=y)
print(data)